# Cleaning-Bank-Marketing-Data

## [ Mini Project ] : Tasked with cleaning marketing campaign data on personal loans to meet specified structure and data types for a PostgreSQL database, enabling storage and easy integration of future campaigns. 

**Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is 
around 10%. This might not sound like a lot, but in September 2022 alone UK consumers borrowed around £1.5 billion, which would mean approximately £300 million in interest generated by banks over 2 years!**

### Simulated situations : 
Asked to work with a bank to clean data collected from a recent marketing campaign aimed at encouraging customers to take out a personal loan. The cleaned data must conform to the specific structure and data types specified by the bank in order to plan future marketing campaigns. This data will be used to set up a PostgreSQL database, which will store the current campaign’s data and allow for the easy import of data from future campaigns.


#### The bank has supplied a CSV file called "bank_marketing.csv," which needs to be cleaned, reformatted, and split into three final CSV files. Specifically, the three files should have the names and contents outlined below:

#### `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

#### `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

#### `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [1]:
import pandas as pd
import numpy as np

#import ["bank_marketing.csv"] file
#Making DataFrame
bm = pd.read_csv("bank_marketing.csv")

#[Client.csv]
client = bm[["client_id", "age", "job", "marital", "education", "credit_default", "mortgage"]]

#cleaning_client
client["education"] = client["education"].str.replace('.','_')
client["education"] = client["education"].replace("unknown",np.nan)
client["job"] = client["education"].str.replace('.','_')

for col in ["credit_default", "mortgage"]:
    client[col] = client[col].map({'yes' : 1,'no' : 0, 'unknown': 0}).astype(bool)
    
#[Campaign.csv]
campaign = bm[["client_id", "number_contacts", "contact_duration", "previous_campaign_contacts", "previous_outcome", "campaign_outcome", "month", "day"]]


#Cleaning_campaign : choose options ;} ; giving same result

#Option: 1
for col in ["campaign_outcome","previous_outcome"]:
    campaign[col] = campaign[col].map({'yes' : 1,'no' : 0, 'success' : 1, 'failure': 0, 'nonexistent': 0}).astype(bool)

ModuleNotFoundError: No module named 'pandas'

### Option: 2 
```campaign["campaign_outcome"] = campaign["campaign_outcome"].map({"yes" : 1,"no": 0}).astype(bool)```
```campaign["previous_outcome"] = campaign["previous_outcome"].map({"success" : 1,"failure": 0,"nonexistent": 0}).astype(bool)```

In [ ]:
#creating new column
campaign["year"] = "2022"
campaign["day"] = campaign["day"].astype(str)
campaign["last_contact_date"] = campaign["year"] + '-' + campaign["month"] + '-' + campaign["day"]
campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], format = "%Y-%b-%d" )

#deleting columns
for col in ["year" , "month" , "day"]:
    campaign.drop(columns = col, inplace = True)

#[Economics.csv]
economics = bm[["client_id", "cons_price_idx", "euribor_three_months"]]


#Saving data
client.to_csv("client.csv", index = False)
campaign.to_csv("campaign.csv", index = False)
economics.to_csv("economics.csv", index = False)